In [1]:
import circuitgraph as cg
import pandas as pd
import networkx as nx
import os
import math
from circuitgraph.io import genus_flops, generic_flop, dc_flops
from circuitgraph import BlackBox

### Making Stdcell library dict

In [2]:
stdcell45nmlib = pd.read_excel("../Data/stdcell45nm.xlsx")

In [157]:
stdcell45nmlib

,MACROS,AREA
0,nand,3.2851
1,or,3.2851
2,buf,3.2851
3,not,3.2851
4,and,3.2851
5,nor,4.6930
6,fflopd,7.0395
7,fflond,7.0385
8,xor,4.2237
9,xnor,4.2237


In [4]:
stdlib45nm_dict = dict(zip(stdcell45nmlib['MACROS'], stdcell45nmlib['AREA']))

In [159]:
# stdlib45nm_dict

## Making Grid Cell Dimensional dict

In [160]:
circuit_area_df = pd.read_excel('../Data/grid_size.xlsx')
circuit_area_df

,CIRCUIT,grid_width,grid_height,total_gates,n_and,n_or,n_nand,n_nor,n_not,n_buf,n_fflopd
0,c17.v,6,6,6,0,0,6,0,0,0,0
1,c3540.v,84,84,1669,498,92,298,68,490,223,0
2,c5315.v,98,98,2307,718,214,454,27,581,313,0
3,c6288.v,110,110,2416,256,0,0,2128,32,0,0
4,c7552.v,120,120,3513,776,244,1028,54,876,535,0
5,s13207.v,80,80,1367,0,0,502,123,262,281,199
6,s38417.v,244,244,13452,0,0,7411,576,2491,1512,1462
7,s38584.v,229,229,11864,0,0,6436,899,2116,1254,1159
8,i2c.v,92,92,2037,1156,32,0,0,834,15,0
9,div.v,637,637,100985,56536,93,0,0,44356,0,0


In [7]:
# (circuit_area_df['grid_height'].where(circuit_area_df.CIRCUIT == 'c17.v'))[0]

In [8]:
circuit_area_dict = dict(zip(circuit_area_df['CIRCUIT'], zip(circuit_area_df['grid_width'], circuit_area_df['grid_height'])))

In [9]:
circuit_information = {row['CIRCUIT']: row.drop('CIRCUIT').to_dict() for _, row in circuit_area_df.iterrows()}

In [41]:
circuit_dict_df = pd.read_excel('../Data/circuit_nodes_info.xlsx')

In [44]:
new_cols = {'Unnamed: 0':'Features'}
circuit_dict_df.rename(columns=new_cols, inplace=True)

In [46]:
circuit_dict_df.set_index(circuit_dict_df['Features'],inplace=True)

In [47]:
circuit_dict_df

,Features,c17.v,c3540.v,c5315.v,c6288.v,c7552.v,s13207.v,s38417.v,s38584.v,i2c.v,voter.v,square.v,multiplier.v,div.v,memctrl.v,sqrt.v
Features,,,,,,,,,,,,,,,,
nodes,nodes,"{'N1', 'N2', 'N22', 'N6', 'N10', 'N19', 'N7', ...","{'N4062', 'N4115', 'N1598', 'N1404', 'N3100', ...","{'N7686', 'N3724', 'N7940', 'N3511', 'N1962', ...","{'N6230', 'N3724', 'N1404', 'N4001', 'N3511', ...","{'N9242', 'N8246', 'N9569', 'N9235', 'N6230', ...","{'w58', 'g1327_reg.D', 'w19', 'g7293', 'n_431'...","{'g322_reg.Q', 'g1867', 'n_1394', 'n_1053', 'n...","{'n_1394', 'n_1053', 'n_2942', 'n_2659', 'g658...","{'po139', 'n_431', 'n320', 'n_1150', 'n1034', ...","{'n_1053', 'n5537', 'n_13652', 'n_6912', 'n519...","{'n_1053', 'n_16243', 'n5537', 'n_13652', 'n_1...","{'n_21799', 'n_1053', 'n_16243', 'n5537', 'n_1...","{'n_26644', 'n_40008', 'n_1053', 'n_16243', 'n...","{'n_26644', 'n_1053', 'n_16243', 'n_14278', 'n...","{'n_1053', 'n_16243', 'n5537', 'n_13652', 'n_1..."
n_and,n_and,set(),"{'N2417', 'N4549', 'N3664', 'N3277', 'N2440', ...","{'N6096', 'N7686', 'N6107', 'N3724', 'N3511', ...","{'N666', 'N879', 'N828', 'N642', 'N1005', 'N73...","{'N8262', 'N4641', 'N3035', 'N1962', 'N1813', ...",set(),set(),set(),"{'n1487', 'n1236', 'n1616', 'po139', 'n960', '...","{'n4761', 'n5537', 'n7757', 'n10975', 'n10248'...","{'n17438', 'n477', 'n4761', 'n5537', 'n7757', ...","{'n5537', 'n5194', 'n23503', 'n12126', 'n22485...","{'n47660', 'n55295', 'n5537', 'n29083', 'n3423...","{'n47660', 'n5537', 'n29083', 'n34231', 'n4088...","{'n5537', 'n12126', 'n22485', 'n20909', 'n2376..."
n_or,n_or,set(),"{'N2697', 'N1991', 'N2769', 'N2659', 'N3415', ...","{'N3849', 'N3861', 'N6566', 'N3745', 'N6925', ...",set(),"{'N9242', 'N9089', 'N3891', 'N8898', 'N2654', ...",set(),set(),set(),"{'po138', 'po123', 'po015', 'po119', 'po079', ...",{'maj'},"{'\\asquared[102]', '\\asquared[66]', '\\asqua...",set(),"{'\\remainder[37]', '\\quotient[2]', '\\remain...","{'po0840', 'po0832', 'po0720', 'po0461', 'po09...","{'\\asqrt[54]', '\\asqrt[12]', '\\asqrt[47]', ..."
n_nand,n_nand,"{'N10', 'N19', 'N23', 'N22', 'N11', 'N16'}","{'N1821', 'N4078', 'N4740', 'N5221', 'N5309', ...","{'N6947', 'N5309', 'N4264', 'N7940', 'N7658', ...",set(),"{'N11236', 'N9569', 'N9235', 'N6540', 'N6107',...","{'w58', 'n_431', 'n_607', 'n_483', 'w106', 'n_...","{'n_1394', 'n_1053', 'n_6912', 'n_1734', 'n_59...","{'n_1394', 'n_1053', 'n_2942', 'n_2659', 'n_69...",set(),set(),set(),set(),set(),set(),set()
n_nor,n_nor,set(),"{'N4255', 'N3394', 'N4258', 'N1870', 'N3399', ...","{'N4575', 'N4646', 'N6553', 'N6872', 'N6494', ...","{'N6230', 'N3724', 'N1404', 'N4001', 'N3511', ...","{'N8134', 'N9149', 'N3658', 'N4871', 'N9146', ...","{'n_438', 'n_109', 'n_117', 'n_309', 'n_501', ...","{'n_6056', 'n_4385', 'n_2858', 'n_8285', 'n_71...","{'n_5168', 'n_5883', 'n_8812', 'n_1894', 'n_60...",set(),set(),set(),"{'n320', 'n3120', 'n24704', 'n22105', 'n23417'...",set(),"{'n_38230', 'n_37626', 'n_38289', 'n_37650', '...",{'n_16617'}
n_not,n_not,set(),"{'N4115', 'N1598', 'N1404', 'N3100', 'N3035', ...","{'N6831', 'N3145', 'N2903', 'N5091', 'N1660', ...","{'N1374', 'N1384', 'N1343', 'N1400', 'N1380', ...","{'N8246', 'N6096', 'N6230', 'N6228', 'N10544',...","{'w211', 'w19', 'w', 'n_49', 'n_305', 'w93', '...","{'n_8047', 'n_2942', 'n_2659', 'w311', 'w19', ...","{'n_1972', 'n_2862', 'w19', 'n_7154', 'n_9185'...","{'n_851', 'n_431', 'n_607', 'n_483', 'n_294', ...","{'n_1394', 'n_1053', 'n_2942', 'n_2659', 'n_13...","{'n_1394', 'n_1053', 'n_2942', 'n_2659', 'n_16...","{'n_21799', 'n_22602', 'n_1394', 'n_1053', 'n_...","{'n_26644', 'n_21799', 'n_40008', 'n_1053', 'n...","{'n_26644', 'n_21799', 'n_1053', 'n_16243', 'n...","{'n_1394', 'n_1053', 'n_2942', 'n_2659', 'n_16..."
n_buf,n_buf,set(),"{'N4062', 'N3965', 'N1384', 'N3472', 'N3142', ...","{'N6331', 'N1384', 'N6370', 'N1433', 'N1606', ...",set(),"{'N1374', 'N8760', 'N3724', 'N1404', 'N1160', ...","{'g4267', 'g7293', 'new_g3635_', 'g1057', 'g10...","{'g1867', 'g2273', 'g1098', 'g780', 'g1358', '...",

In [62]:
circuit_dict = circuit_dict_df.to_dict()
del circuit_dict['Features']

In [69]:
class ChipDesignGraph:
    def __init__(self,circuit_name):
        
        self.circuit = cg.from_file(f'../Data/Circuits/{circuit_name}', blackboxes= genus_flops + dc_flops)
        self.circuit_name = self.circuit.name+'.v'
        self.grid_size = (circuit_information[self.circuit_name]['grid_width'], circuit_information[self.circuit_name]['grid_height'] )
        self.design_graph = nx.grid_2d_graph(*self.grid_size)
        self.node_attributes = {}
        self.edge_attributes = {}
    

    def get_circuit(self):
        return self.circuit  

    
        

    def add_node_attributes(self,  node_name_in):
        """
        Add attributes to each node in the graph.
        node_data: Dictionary mapping node IDs to attribute dictionaries.
        """

        node_data = {node_name_in: self.circuit.type(node_name_in)}
        
        self.node_attributes.update(node_data)
        for node, data in node_data.items():
            self.design_graph.nodes[node].update(data)

    
    def add_edge_attributes(self, edge_data):
        """
        Add attributes to edges in the graph.
        edge_data: Dictionary mapping edge tuples (u, v) to attribute dictionaries.
        """
        self.edge_attributes.update(edge_data)
        for edge, data in edge_data.items():
            self.design_graph.edges[edge].update(data)

    
    def DrawGraph(self):
        return nx.draw_spectral(self.design_graph)
        
        
    

In [70]:
def add_to_dictionary(circuit_name):
    aman = ChipDesignGraph(f'../Data/Circuits/{circuit_name}')
    c = aman.get_circuit()
    
    ls_and = set()
    ls_or = set()
    ls_nand = set()
    ls_nor = set()
    ls_not = set()
    ls_buf = set()
    ls_fflopd = set()
    
    
    for node in c.nodes():
        if c.type(node) == 'and':
            ls_and.add(node)
        if c.type(node) == 'or':
            ls_or.add(node)
        if c.type(node) == 'nand':
            ls_nand.add(node)
        if c.type(node) == 'nor':
            ls_nor.add(node)
        if c.type(node) == 'not':
            ls_not.add(node)
        if c.type(node) == 'buf':
            ls_buf.add(node)
        if c.type(node) == 'bb_input':
            if '_reg.D' in node:
                ls_fflopd.add(node)
            
        
    cr = {circuit_name : {
        'nodes' : c.nodes(),
        'n_and' : ls_and,
        'n_or' : ls_or,
        'n_nand' : ls_nand,
        'n_nor' : ls_nor,
        'n_not' : ls_not,
        'n_buf' : ls_buf,
        'n_fflopd' : ls_fflopd,
    }}
    
    circuit_dict.update(cr)
    circuit_dict

In [71]:
# add_to_dictionary('c17.v')
# circuit_dict['c17.v']['n_nand']

In [130]:
chipgraph_1 = ChipDesignGraph('c17.v')

In [131]:
circuit_1 = chipgraph_1.get_circuit()

In [132]:
# dir(circuit_1)

In [133]:
# circuit_1.edges()

In [134]:
# circuit_1.is_cyclic()

In [135]:
# x = list(circuit_1.paths('N20','N1947'))

In [136]:
# x

In [137]:
# circuit_1.outputs()

In [138]:
# circuit_1.outputs()

In [149]:
print(len(circuit_1.edges()))
print(circuit_1.edges())

12
{('N11', 'N19'), ('N1', 'N10'), ('N7', 'N19'), ('N3', 'N10'), ('N3', 'N11'), ('N10', 'N22'), ('N16', 'N22'), ('N16', 'N23'), ('N19', 'N23'), ('N2', 'N16'), ('N6', 'N11'), ('N11', 'N16')}


In [141]:
circuit_1.nodes()

{'N1', 'N10', 'N11', 'N16', 'N19', 'N2', 'N22', 'N23', 'N3', 'N6', 'N7'}

In [142]:
circuit_1.inputs()

{'N1', 'N2', 'N3', 'N6', 'N7'}

In [143]:
circuit_1.outputs()

{'N22', 'N23'}

In [156]:
circuit_1.type(['N10','N22'])

['nand', 'nand']